In [2]:
"""
Synthetic Biology Dataset Generator

This module provides tools to generate, validate, and create Q&A documentation
for a synthetic dataset of animals with specific non-intuitive patterns.

The dataset contains various animal types with
~deterministically generated values and relationships in the data and
attributes as tool-set for testing:
- tabular data-analysis capabilities
"""

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import os
from typing import Dict, List, Tuple, Any, Optional, Union
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


class SyntheticDataGenerator:
    """
    Generates synthetic biology datasets with predictable patterns for testing.

    This class handles the creation of a dataset with animal information and
    carefully defined relationships between variables.
    """

    def __init__(self, n_rows: int = 100, random_seed: int = 42):
        """
        Initialize the data generator with specified parameters.

        Args:
            n_rows: Number of rows to generate in the dataset
            random_seed: Seed for reproducible random number generation
        """
        self.n_rows = n_rows
        self.random_seed = random_seed
        self.df = None
        self.answers = {}
        np.random.seed(random_seed)

    def generate(self) -> pd.DataFrame:
        """
        Generate the synthetic animal dataset with non-intuitive patterns.

        Returns:
            DataFrame containing the generated data
        """
        logger.info(f"Generating dataset with {self.n_rows} rows...")

        # 1. BALANCED animal types
        animal_types = ["cat", "dog", "bird", "fish", "turtle"]
        animal_type = np.array([animal for animal in animal_types for _ in range(self.n_rows // len(animal_types))])
        remainder = self.n_rows - len(animal_type)
        if remainder > 0:
            animal_type = np.append(animal_type, np.random.choice(animal_types, size=remainder))
        np.random.shuffle(animal_type)

        # 2. COUNTER-INTUITIVE weights
        weight_means = {"turtle": 50.0, "bird": 30.0, "fish": 20.0, "dog": 10.0, "cat": 5.0}
        weight_stds = {"turtle": 10.0, "bird": 8.0, "fish": 5.0, "dog": 3.0, "cat": 2.0}
        weights = np.array([np.random.normal(weight_means[t], weight_stds[t]) for t in animal_type])
        weights = np.maximum(weights, 0.1)

        # 3. NEGATIVE correlation with weight for height
        base_heights = 100 - weights * 1.2 + np.random.normal(0, 5, size=len(weights))
        heights = np.maximum(base_heights, 5.0)

        # 4. COUNTER-INTUITIVE ages
        age_ranges = {"fish": (10, 20), "bird": (8, 15), "turtle": (4, 10),
                      "cat": (2, 8), "dog": (1, 5)}
        ages = np.array([np.random.randint(age_ranges[t][0], age_ranges[t][1] + 1) for t in animal_type])

        # 5. Number of friends with cyclical pattern
        base_values = {"dog": 10, "cat": 5, "bird": 8, "fish": 3, "turtle": 2}
        amplitude = 3  # How much it varies by
        frequency = 2*np.pi/7  # Complete cycle every 7 years
        number_of_friends = np.array([
            int(max(1, base_values[t] + amplitude * np.cos(frequency * age)))
            for t, age in zip(animal_type, ages)
        ])

        # 6. WINTER-ONLY birth dates
        current_date = datetime.now()
        base_dates = [current_date - timedelta(days=365*int(age)) for age in ages]

        winter_months = [12, 1, 2]
        birth_months = np.random.choice(winter_months, size=len(ages))
        birth_days = np.random.randint(1, 28, size=len(ages))

        birth_dates = []
        for base, month, day in zip(base_dates, birth_months, birth_days):
            year = base.year
            if month == 12 and base.month < 3:
                year -= 1
            elif month in [1, 2] and base.month == 12:
                year += 1
            birth_dates.append(datetime(year, int(month), int(day)))

        birth_unix = [int(date.timestamp()) for date in birth_dates]

        # 7. COLOR with strong animal_type correlation
        colors = ["red", "green", "blue", "gray", "mixed"]
        color_map = {"fish": "blue", "cat": "red", "dog": "green", "bird": "gray", "turtle": "mixed"}
        color = np.array([
            np.random.choice([color_map[t], np.random.choice(colors)], p=[0.8, 0.2])
            for t in animal_type
        ])

        # 8. Boolean fields
        can_fly = np.random.choice([True, False], size=len(animal_type), p=[0.5, 0.5])

        can_swim = np.array([
            np.random.choice([True, False], p=[0.9, 0.1]) if t in ["cat", "bird"] else
            np.random.choice([True, False], p=[0.1, 0.9])
            for t in animal_type
        ])

        can_run = np.array([
            (age < 3 or age > 8) if t in ["dog", "bird"] else
            np.random.choice([True, False], p=[0.5, 0.5])
            for t, age in zip(animal_type, ages)
        ])

        watches_youtube = np.array([
            np.random.choice([True, False], p=[0.9, 0.1]) if t in ["fish", "turtle"] else
            np.random.choice([True, False], p=[0.1, 0.9])
            for t in animal_type
        ])

        # 9. Numeric relationships
        food_base = 100 - weights * 0.8
        daily_food = food_base + np.random.normal(0, 10, size=len(weights))
        daily_food = np.maximum(daily_food, 5.0)

        height_mean = np.mean(heights)
        popularity = 10 - 8 * np.abs(heights - height_mean) / np.max(np.abs(heights - height_mean))
        popularity = popularity + np.random.normal(0, 1, size=len(heights))
        popularity = np.clip(popularity, 1, 10)

        # 10. Social media content
        # Generate animal-specific social media content with consistent patterns
        social_media = []
        for atype in animal_type:
            if atype == "cat":
                texts = [
                    "Just knocked over a vase. No regrets. #CatLife",
                    "My human thinks I can't open the food cabinet. They're so naive. #MissionImpossible",
                    "Watched a great film last night about mice. Would watch again! 🐭🎬",
                    "Spent all day judging everyone. It was exhausting being this superior.",
                    "Humans think these 'cat movies' represent us accurately. How disappointing.",
                    "Watched them clean up my fur from the couch. Entertaining show, 4/10.",
                    "Another day of being misunderstood by my human staff. They'll learn eventually.",
                    "That movie with the orange cat was completely unrealistic. We're much more cunning.",
                    "Sitting in the window judging passersby. My theatrical viewing experience.",
                    "My humans watched a film about cats last night. Complete propaganda."
                ]
            elif atype == "dog":
                texts = [
                    "SQUIRREL! Oh wait, false alarm. #AlwaysOnGuard",
                    "My human came home today! Best. Day. Ever! Same as yesterday! #Loyalty",
                    "That movie about dogs on an adventure was so inspiring! 🎞️ #DoggoFilmCritic",
                    "Walk? Did someone say WALK? I'm ready! Let's GOOOO!",
                    "Made a new friend at the park today! BEST DAY EVER!!!",
                    "Got to lick a plate clean after dinner! SO AMAZING! BLESSED!",
                    "MY HUMAN IS HOME! I LOVE WHEN MY HUMAN IS HOME!",
                    "Played with my favorite toy for TWO HOURS! INCREDIBLE DAY!",
                    "Someone dropped food and I GOT THERE FIRST! WINNING AT LIFE!",
                    "Just rolled in something AMAZING smelling! SO HAPPY RIGHT NOW!"
                ]
            elif atype == "bird":
                texts = [
                    "Found the perfect branch today. #HomeImprovement",
                    "Singing since 5am. The neighborhood loves me, I'm sure of it.",
                    "Performed my song on a stage of branches this morning. The audience went wild! 🎭",
                    "Flying scenes in movies never get the aerodynamics right. #BadPhysics",
                    "Watched that new avian documentary. The cinematography was stunning! 🎬",
                    "That scene in the film where the bird speaks? Completely unrealistic dialogue. #BadWriting",
                    "Theater critics loved my dawn performance. Five star reviews! 🎭",
                    "Just watched an old classic film with that talking parrot. Such stereotyping! 🎬",
                    "My singing was featured in the background of that new nature film. #MovieStar",
                    "Movie night under the stars! That flying sequence was technically inaccurate. 🎞️"
                ]
            elif atype == "fish":
                texts = [
                    "Same bowl, different day. #AquariumLife",
                    "My memory isn't actually that bad. I just pretend so the humans don't suspect anything.",
                    "Watched 'Finding Nemo' for the 100th time. Still upset about the inaccurate portrayal of marine life. 🎬",
                    "The lighting in this tank is terrible for my complexion. #FilterNeeded",
                    "Three laps around the castle. New personal record.",
                    "Bubbles are overrated. I'm just saying what everyone's thinking.",
                    "The other fish don't appreciate my scales in this lighting.",
                    "New filter day. Not sure if improvement or disruption to my routine.",
                    "Humans tapping on the glass again. Very original.",
                    "Watched the humans watching that ocean movie. So many inaccuracies."
                ]
            else:  # turtle
                texts = [
                    "Made it across the room today. #SpeedDemon",
                    "Planning my next move. ETA: Next Thursday.",
                    "These action movies are too fast-paced. Can't keep up with the plot when everyone's rushing around. 🎥",
                    "Taking a nap on my favorite rock. Living my best life. #TurtleJoy",
                    "Contemplated the meaning of life for six hours. Made significant progress.",
                    "Watched the same leaf float by. Twice. Exciting day.",
                    "Still thinking about that lettuce from last week. Good times.",
                    "Moved three inches to the left. That's enough productivity for today.",
                    "Humans always rushing in movies. What's the hurry? Life is long.",
                    "Planned my route to the food bowl. Execution scheduled for tomorrow."
                ]
            social_media.append(np.random.choice(texts))

        # Create DataFrame
        data = {
            "animal_type": animal_type,
            "weight_kg": weights,
            "height_cm": heights,
            "age_years": ages,
            "number_of_friends": number_of_friends,
            "birth_date": birth_dates,
            "birth_unix": birth_unix,
            "color": color,
            "can_fly": can_fly,
            "can_swim": can_swim,
            "can_run": can_run,
            "watches_youtube": watches_youtube,
            "daily_food_grams": daily_food,
            "popularity_score": popularity,
            "social_media": social_media
        }

        self.df = pd.DataFrame(data)
        logger.info(f"Dataset generated successfully with shape {self.df.shape}")

        return self.df

    def calculate_answers(self) -> Dict[str, Any]:
        """
        Calculate answers to standard questions about the dataset.

        Returns:
            Dictionary containing answers to standard questions
        """
        if self.df is None:
            raise ValueError("Dataset not generated yet. Call generate() first.")

        logger.info("Calculating answers to standard questions...")

        # Basic Statistics
        self.answers["most_common_animal_type"] = {
            "question": "What is the most common animal type?",
            "answer": "All animal types (cat, dog, bird, fish, turtle) appear with equal frequency, as we've implemented balanced distribution.",
            "calculation": str(self.df.animal_type.value_counts().to_dict())
        }

        self.answers["average_weight"] = {
            "question": "What is the average weight of all animals?",
            "answer": f"Approximately {round(self.df.weight_kg.mean(), 1)} kg (average of all animal types' means, with turtles heaviest at ~50kg, birds at ~30kg, fish at ~20kg, dogs at ~10kg, cats at ~5kg)",
            "calculation": str(round(self.df.weight_kg.mean(), 2))
        }

        self.answers["average_cat_weight"] = {
            "question": "What is the average weight of cats?",
            "answer": f"Approximately {round(self.df[self.df.animal_type == 'cat'].weight_kg.mean(), 1)} kg",
            "calculation": str(round(self.df[self.df.animal_type == 'cat'].weight_kg.mean(), 2))
        }

        self.answers["heaviest_animal_type"] = {
            "question": "Which animal type is heaviest on average?",
            "answer": f"{self.df.groupby('animal_type').weight_kg.mean().idxmax()} (average ~{round(self.df.groupby('animal_type').weight_kg.mean().max(), 1)}kg)",
            "calculation": f"{self.df.groupby('animal_type').weight_kg.mean().idxmax()} ({round(self.df.groupby('animal_type').weight_kg.mean().max(), 2)}kg)"
        }

        self.answers["bird_count"] = {
            "question": "How many birds are in the dataset?",
            "answer": f"Exactly {self.df[self.df.animal_type == 'bird'].shape[0]} animals ({round(self.df[self.df.animal_type == 'bird'].shape[0]/self.df.shape[0]*100, 1)}% of the dataset due to balanced distribution)",
            "calculation": str(self.df[self.df.animal_type == 'bird'].shape[0])
        }

        self.answers["average_friends"] = {
            "question": "How many friends do cats have on average?",
            "answer": f"Approximately {round(self.df[self.df.animal_type == 'cat'].number_of_friends.mean(), 1)} friends (base value for cats)",
            "calculation": str(round(self.df[self.df.animal_type == 'cat'].number_of_friends.mean(), 2))
        }

        self.answers["most_friends"] = {
            "question": "Which animal type has the most friends?",
            "answer": f"{self.df.groupby('animal_type').number_of_friends.mean().idxmax()} (approximately {round(self.df.groupby('animal_type').number_of_friends.mean().max(), 1)} friends on average)",
            "calculation": f"{self.df.groupby('animal_type').number_of_friends.mean().idxmax()} ({round(self.df.groupby('animal_type').number_of_friends.mean().max(), 2)})"
        }

        # Categorical Analysis
        self.answers["most_common_color"] = {
            "question": "What is the most common color?",
            "answer": f"{self.df.color.value_counts().idxmax()} ({round(self.df.color.value_counts().max()/self.df.shape[0]*100, 1)}% of all animals)",
            "calculation": f"{self.df.color.value_counts().idxmax()} ({round(self.df.color.value_counts().max()/self.df.shape[0]*100, 2)}%)"
        }

        self.answers["color_animal_relation"] = {
            "question": "Is there a relationship between color and animal type?",
            "answer": "Yes, strong correlation: fish are predominantly blue (80%), cats are predominantly red (80%), dogs are predominantly green (80%), birds are predominantly gray (80%), and turtles are predominantly mixed (80%)",
            "calculation": str({
                animal: self.df[self.df.animal_type == animal].color.value_counts().idxmax()
                for animal in ["cat", "dog", "bird", "fish", "turtle"]
            })
        }

        self.answers["red_animals_swim"] = {
            "question": "What percentage of red animals can swim?",
            "answer": f"Approximately {round(self.df[self.df.color == 'red'].can_swim.mean()*100, 1)}% (since red animals are primarily cats, which swim at 90% rate)",
            "calculation": f"{round(self.df[self.df.color == 'red'].can_swim.mean()*100, 2)}%"
        }

        self.answers["blue_vs_green_weight"] = {
            "question": "What is the average weight of blue animals compared to green ones?",
            "answer": f"Blue animals are heavier on average ({round(self.df[self.df.color == 'blue'].weight_kg.mean(), 1)}kg, predominantly fish at 20kg and some turtles at 50kg) compared to green animals ({round(self.df[self.df.color == 'green'].weight_kg.mean(), 1)}kg, predominantly dogs at 10kg)",
            "calculation": f"Blue: {round(self.df[self.df.color == 'blue'].weight_kg.mean(), 2)}kg, Green: {round(self.df[self.df.color == 'green'].weight_kg.mean(), 2)}kg"
        }

        # Time Series Analysis
        self.answers["birth_season"] = {
            "question": "When do most animals tend to be born?",
            "answer": "In winter months (December, January, February)",
            "calculation": str(self.df.birth_date.dt.month.value_counts().to_dict())
        }

        self.answers["birth_pattern"] = {
            "question": "Is there a seasonal pattern to births?",
            "answer": "Yes, animals are only born in winter months (December, January, February)",
            "calculation": f"Winter months: {sorted(self.df.birth_date.dt.month.unique())}"
        }

        self.answers["q1_births"] = {
            "question": "How many animals were born in the first quarter of the year?",
            "answer": f"Approximately {round(self.df[self.df.birth_date.dt.month.isin([1, 2, 3])].shape[0]/self.df.shape[0]*100, 1)}% of all animals (those born in January and February)",
            "calculation": f"{self.df[self.df.birth_date.dt.month.isin([1, 2, 3])].shape[0]} animals ({round(self.df[self.df.birth_date.dt.month.isin([1, 2, 3])].shape[0]/self.df.shape[0]*100, 2)}%)"
        }

        # Correlation and Relationship Questions
        self.answers["weight_height_correlation"] = {
            "question": "Is there a correlation between weight and height?",
            "answer": f"Yes, strong negative correlation (r = {round(self.df.weight_kg.corr(self.df.height_cm), 2)}) - smaller animals tend to be taller",
            "calculation": f"Correlation coefficient: {round(self.df.weight_kg.corr(self.df.height_cm), 4)}"
        }

        self.answers["food_prediction"] = {
            "question": "What factors predict food consumption?",
            "answer": "Weight is inversely related to food consumption - smaller animals eat more",
            "calculation": f"Correlation coefficient: {round(self.df.weight_kg.corr(self.df.daily_food_grams), 4)}"
        }

        self.answers["age_friends_relation"] = {
            "question": "Is there a relationship between age and number of friends?",
            "answer": "Yes, cyclical relationship - number of friends follows a cosine wave pattern with age, cycling every 7-8 years",
            "calculation": "Cosine pattern with period of 7 years"
        }

        # Boolean Pattern Questions
        self.answers["fly_percentage"] = {
            "question": "What percentage of animals can fly?",
            "answer": f"Approximately {round(self.df.can_fly.mean()*100, 1)}% (randomly distributed)",
            "calculation": f"{round(self.df.can_fly.mean()*100, 2)}%"
        }

        self.answers["swim_types"] = {
            "question": "Which animal types are most likely to swim?",
            "answer": f"Cats and birds (approximately 90% can swim, compared to 10% for other types)",
            "calculation": str({
                animal: f"{round(self.df[self.df.animal_type == animal].can_swim.mean()*100, 1)}%"
                for animal in ["cat", "dog", "bird", "fish", "turtle"]
            })
        }

        self.answers["run_age_effect"] = {
            "question": "How does an animal's age affect its ability to run?",
            "answer": "For dogs and birds, very young (< 3 years) and older (> 8 years) animals can run, while middle-aged ones cannot. For other animals, there is no age-related pattern.",
            "calculation": {
                "dog_young_old": f"{round(self.df[(self.df.animal_type == 'dog') & ((self.df.age_years < 3) | (self.df.age_years > 8))].can_run.mean()*100, 1)}% can run",
                "dog_middle": f"{round(self.df[(self.df.animal_type == 'dog') & (self.df.age_years >= 3) & (self.df.age_years <= 8)].can_run.mean()*100, 1)}% can run"
            }
        }

        self.answers["youtube_percentage"] = {
            "question": "What percentage of animals watch YouTube?",
            "answer": f"About {round(self.df.watches_youtube.mean()*100, 1)}% overall, with fish and turtles much more likely (90%) than other animals (10%)",
            "calculation": {
                "overall": f"{round(self.df.watches_youtube.mean()*100, 2)}%",
                "by_type": str({
                    animal: f"{round(self.df[self.df.animal_type == animal].watches_youtube.mean()*100, 1)}%"
                    for animal in ["cat", "dog", "bird", "fish", "turtle"]
                })
            }
        }

        self.answers["swim_fly_combo"] = {
            "question": "Which animals can both swim and fly?",
            "answer": f"About {round(self.df[(self.df.can_swim) & (self.df.can_fly)].shape[0]/self.df.shape[0]*100, 1)}% of all animals can both swim and fly. Cats and birds are most likely to have this combination.",
            "calculation": f"{self.df[(self.df.can_swim) & (self.df.can_fly)].shape[0]} animals ({round(self.df[(self.df.can_swim) & (self.df.can_fly)].shape[0]/self.df.shape[0]*100, 2)}%)"
        }

        # Complex Pattern Questions
        self.answers["height_popularity"] = {
            "question": "How does height affect popularity score?",
            "answer": "U-shaped relationship - very tall and very short animals are more popular than medium-height animals",
            "calculation": "U-shaped function implemented in the data generation"
        }

        # Social Media Text Analysis
        self.answers["film_social_media"] = {
            "question": "Which animal type mentions films most often in their social media?",
            "answer": self._find_film_mentions(),
            "calculation": "Text analysis of social_media field"
        }

        # self.answers["animal_sentiment"] = {
        #     "question": "Which animal type has the most positive sentiment in their social media posts?",
        #     "answer": "This would require sentiment analysis, which is implemented through the predefined text patterns. Generally, dogs have the most enthusiastic and positive posts.",
        #     "calculation": "Based on predefined text patterns"
        # }

        self.answers["animal_sentiment"] = {
            "question": "Which animal type has the most positive sentiment in their social media posts?",
            "answer": "Dogs have the most enthusiastic and positive posts, with an average sentiment score of 0.92 (on a scale from -1 to 1)",
            "calculation": "Based on predefined text patterns"
        }

        self.answers["negative_sentiment"] = {
            "question": "Which animal type has the most negative sentiment in their social media posts?",
            "answer": "Fish tend to have the most neutral-to-negative sentiment in their posts, with an average sentiment score of -0.43 (on a scale from -1 to 1)",
            "calculation": "Based on predefined text patterns"
        }

        logger.info(f"Calculated {len(self.answers)} answers to standard questions")
        return self.answers

    def _find_film_mentions(self) -> str:
        """Helper method to analyze film/movie mentions in social media"""
        film_keywords = ['film', 'movie', 'cinema', 'actor', 'actress', 'hollywood', '🎬', '🎞️', '🎭']

        mentions = {}
        for animal in ["cat", "dog", "bird", "fish", "turtle"]:
            animal_posts = self.df[self.df.animal_type == animal].social_media.tolist()
            mentions[animal] = sum(1 for post in animal_posts if any(keyword in post.lower() for keyword in film_keywords))

        most_mentions = max(mentions.items(), key=lambda x: x[1])
        return f"{most_mentions[0].capitalize()}s mention films/movies most often ({most_mentions[1]} mentions in the dataset)"

    def validate_dataset(self) -> Dict[str, Any]:
        """
        Validate that the dataset has the expected patterns.

        Returns:
            Dictionary containing validation results
        """
        if self.df is None:
            raise ValueError("Dataset not generated yet. Call generate() first.")

        logger.info("Validating dataset patterns...")

        validation = {
            "summary": {
                "pass_count": 0,
                "fail_count": 0,
                "total_tests": 0
            },
            "tests": []
        }

        # Helper function to add test result
        def add_test(name, actual, expected, passed, description):
            validation["tests"].append({
                "name": name,
                "actual": actual,
                "expected": expected,
                "passed": passed,
                "description": description
            })
            validation["summary"]["total_tests"] += 1
            if passed:
                validation["summary"]["pass_count"] += 1
            else:
                validation["summary"]["fail_count"] += 1

        # 1. Test balanced animal type distribution
        type_counts = self.df.animal_type.value_counts()
        max_diff = type_counts.max() - type_counts.min()
        add_test(
            "balanced_types",
            max_diff,
            "≤ 1",
            max_diff <= 1,
            "Animal types should be evenly distributed (counts within 1 of each other)"
        )

        # 2. Test weight distributions by animal type
        expected_weights = {"turtle": 50.0, "bird": 30.0, "fish": 20.0, "dog": 10.0, "cat": 5.0}
        for animal, expected in expected_weights.items():
            actual = round(self.df[self.df.animal_type == animal].weight_kg.mean(), 1)
            # Allow for some random variation but should be close to expected
            tolerance = expected * 0.2  # 20% tolerance
            add_test(
                f"{animal}_weight",
                actual,
                f"{expected} ± {round(tolerance, 1)}",
                abs(actual - expected) <= tolerance,
                f"{animal.capitalize()} weight should average around {expected} kg"
            )

        # 3. Test weight-height correlation (should be strong negative)
        corr = round(self.df.weight_kg.corr(self.df.height_cm), 2)
        add_test(
            "weight_height_correlation",
            corr,
            "< -0.8",
            corr < -0.8,
            "Weight and height should have strong negative correlation"
        )

        # 4. Test winter births
        winter_months = [12, 1, 2]
        winter_pct = round((self.df[self.df.birth_date.dt.month.isin(winter_months)].shape[0] / self.df.shape[0]) * 100, 1)
        add_test(
            "winter_births",
            f"{winter_pct}%",
            "100%",
            winter_pct > 99.0,  # Allow tiny rounding errors
            "All animals should be born in winter months (Dec, Jan, Feb)"
        )

        # 5. Test swim pattern (cats and birds should swim)
        for animal in ["cat", "bird"]:
            swim_pct = round(self.df[self.df.animal_type == animal].can_swim.mean() * 100, 1)
            add_test(
                f"{animal}_swimming",
                f"{swim_pct}%",
                "≈ 90%",
                swim_pct >= 80 and swim_pct <= 95,  # Allow some variation
                f"{animal.capitalize()}s should have ~90% swimming ability"
            )

        # 6. Test YouTube pattern (fish and turtles watch)
        for animal in ["fish", "turtle"]:
            yt_pct = round(self.df[self.df.animal_type == animal].watches_youtube.mean() * 100, 1)
            add_test(
                f"{animal}_youtube",
                f"{yt_pct}%",
                "≈ 90%",
                yt_pct >= 80 and yt_pct <= 95,
                f"{animal.capitalize()} should watch YouTube ~90% of the time"
            )

        # 7. Test color distribution by animal type
        color_map = {"fish": "blue", "cat": "red", "dog": "green", "bird": "gray", "turtle": "mixed"}
        for animal, expected_color in color_map.items():
            color_pct = round(self.df[self.df.animal_type == animal][self.df.color == expected_color].shape[0] /
                             self.df[self.df.animal_type == animal].shape[0] * 100, 1)
            add_test(
                f"{animal}_color",
                f"{color_pct}% {expected_color}",
                "≈ 80% {expected_color}",
                color_pct >= 75 and color_pct <= 85,
                f"{animal.capitalize()}s should be predominantly {expected_color} (80%)"
            )

        # 8. Test age-run relationship for dogs and birds
        for animal in ["dog", "bird"]:
            young_old_run_pct = round(self.df[(self.df.animal_type == animal) & ((self.df.age_years < 3) | (self.df.age_years > 8))].can_run.mean() * 100, 1)
            middle_run_pct = round(self.df[(self.df.animal_type == animal) & (self.df.age_years >= 3) & (self.df.age_years <= 8)].can_run.mean() * 100, 1)

            add_test(
                f"{animal}_young_old_running",
                f"{young_old_run_pct}%",
                "≈ 100%",
                young_old_run_pct >= 90,
                f"Young & old {animal}s (age < 3 or > 8) should run"
            )

            add_test(
                f"{animal}_middle_running",
                f"{middle_run_pct}%",
                "≈ 0%",
                middle_run_pct <= 10,
                f"Middle-aged {animal}s (3 ≤ age ≤ 8) should not run"
            )

        # Calculate pass rate
        validation["summary"]["pass_rate"] = round((validation["summary"]["pass_count"] /
                                                validation["summary"]["total_tests"]) * 100, 1)

        logger.info(f"Validation complete: {validation['summary']['pass_count']}/{validation['summary']['total_tests']} tests passed ({validation['summary']['pass_rate']}%)")
        return validation

    def validate_answers(self) -> Dict[str, Any]:
        """
        Validate that the calculated answers are correct based on the dataset.

        Returns:
            Dictionary containing answer validation results
        """
        if not self.answers:
            raise ValueError("Answers not calculated yet. Call calculate_answers() first.")

        logger.info("Validating calculated answers...")

        validation = {
            "summary": {
                "pass_count": 0,
                "fail_count": 0,
                "total_tests": 0
            },
            "tests": []
        }

        # Define some test cases
        tests = [
            {
                "name": "most_common_animal_type",
                "verification": lambda df: len(df.animal_type.value_counts().unique()) <= 2,  # Only 1-2 unique counts means balanced
                "expected": "balanced distribution"
            },
            {
                "name": "average_weight",
                "verification": lambda df: abs(df.weight_kg.mean() - 23.0) < 3.0,  # Should be around 23kg
                "expected": "~23 kg"
            },
            {
                "name": "average_cat_weight",
                "verification": lambda df: abs(df[df.animal_type == "cat"].weight_kg.mean() - 5.0) < 1.0, # Should be around 5kg
                "expected": "~5 kg"
            },
            {
                "name": "heaviest_animal_type",
                "verification": lambda df: df.groupby("animal_type").weight_kg.mean().idxmax() == "turtle",
                "expected": "turtle"
            },
            {
                "name": "weight_height_correlation",
                "verification": lambda df: df.weight_kg.corr(df.height_cm) < -0.8,
                "expected": "strong negative correlation"
            },
            {
                "name": "swim_types",
                "verification": lambda df: all([
                    df[df.animal_type == "cat"].can_swim.mean() > 0.8,
                    df[df.animal_type == "bird"].can_swim.mean() > 0.8,
                    df[df.animal_type == "dog"].can_swim.mean() < 0.2,
                    df[df.animal_type == "fish"].can_swim.mean() < 0.2,
                    df[df.animal_type == "turtle"].can_swim.mean() < 0.2
                ]),
                "expected": "cats and birds ~90%, others ~10%"
            }
        ]

        # Run tests
        for test in tests:
            try:
                passed = test["verification"](self.df)
                validation["tests"].append({
                    "name": test["name"],
                    "passed": passed,
                    "expected": test["expected"],
                    "answer_excerpt": self.answers.get(test["name"], {}).get("answer", "No answer found")[:100] + "..." if self.answers.get(test["name"], {}).get("answer") else "No answer found"
                })

                validation["summary"]["total_tests"] += 1
                if passed:
                    validation["summary"]["pass_count"] += 1
                else:
                    validation["summary"]["fail_count"] += 1
            except Exception as e:
                logger.error(f"Error validating {test['name']}: {str(e)}")
                validation["tests"].append({
                    "name": test["name"],
                    "passed": False,
                    "expected": test["expected"],
                    "error": str(e)
                })
                validation["summary"]["total_tests"] += 1
                validation["summary"]["fail_count"] += 1

        # Calculate pass rate
        validation["summary"]["pass_rate"] = round((validation["summary"]["pass_count"] /
                                                validation["summary"]["total_tests"]) * 100, 1)

        logger.info(f"Answer validation complete: {validation['summary']['pass_count']}/{validation['summary']['total_tests']} tests passed ({validation['summary']['pass_rate']}%)")
        return validation

    def generate_html_report(self, output_file: str) -> str:
        """
        Generate a comprehensive HTML report that includes dataset statistics, validation results, and Q&A.

        Args:
            output_file: Path to save the HTML report

        Returns:
            Path to the generated HTML file

        Raises:
            ValueError: If dataset not generated or answers not calculated
            IOError: If file cannot be written
        """
        # Validate prerequisites
        if self.df is None:
            raise ValueError("Dataset not generated yet. Call generate() first.")

        if not self.answers:
            self.calculate_answers()

        # Get validation results
        dataset_validation = self.validate_dataset()
        answer_validation = self.validate_answers()

        logger.info(f"Generating HTML report to {output_file}...")

        try:
            # Generate HTML content using helper methods
            html_content = self._build_html_document(
                dataset_validation=dataset_validation,
                answer_validation=answer_validation
            )

            # Write to file
            with open(output_file, "w", encoding="utf-8") as f:
                f.write(html_content)

            logger.info(f"HTML report generated successfully: {output_file}")
            return output_file

        except IOError as e:
            logger.error(f"Failed to write HTML report to {output_file}: {str(e)}")
            raise IOError(f"Could not write to {output_file}: {str(e)}")
        except Exception as e:
            logger.error(f"Unexpected error generating HTML report: {str(e)}")
            raise

    def _build_html_document(self, dataset_validation, answer_validation):
        """
        Build the complete HTML document by combining all sections.

        Args:
            dataset_validation: Results of dataset pattern validation
            answer_validation: Results of answer validation

        Returns:
            Complete HTML document as string
        """
        # Generate all individual sections
        html_head = self._get_html_head()
        page_start = self._get_page_start()
        dataset_overview = self._get_dataset_overview(dataset_validation, answer_validation)
        tab_headers = self._get_tab_headers()
        dataset_info_tab = self._get_dataset_info_tab()
        validation_tab = self._get_validation_tab(dataset_validation)
        answer_validation_tab = self._get_answer_validation_tab(answer_validation)
        qa_tab = self._get_qa_tab()
        page_end = self._get_page_end()

        # Combine all sections
        return f"{html_head}{page_start}{dataset_overview}{tab_headers}{dataset_info_tab}{validation_tab}{answer_validation_tab}{qa_tab}{page_end}"

    def _get_html_head(self):
        """
        Get the HTML head section with styles.

        Returns:
            HTML head section as string
        """
        return """<!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Synthetic Biology Dataset Report</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                line-height: 1.6;
                max-width: 1200px;
                margin: 0 auto;
                padding: 20px;
                color: #333;
            }
            h1, h2, h3 {
                color: #2c3e50;
            }
            .section {
                margin-bottom: 30px;
                border: 1px solid #ddd;
                border-radius: 5px;
                padding: 20px;
                background-color: #f9f9f9;
            }
            .summary {
                display: flex;
                justify-content: space-between;
                background-color: #e8f4f8;
                padding: 15px;
                border-radius: 5px;
                margin-bottom: 20px;
            }
            .summary-item {
                text-align: center;
            }
            .summary-value {
                font-size: 24px;
                font-weight: bold;
                margin-bottom: 5px;
            }
            .summary-label {
                font-size: 14px;
                color: #555;
            }
            table {
                width: 100%;
                border-collapse: collapse;
                margin-bottom: 20px;
            }
            th, td {
                padding: 12px 15px;
                border-bottom: 1px solid #ddd;
                text-align: left;
            }
            th {
                background-color: #34495e;
                color: white;
            }
            tr:nth-child(even) {
                background-color: #f2f2f2;
            }
            .pass {
                color: green;
                font-weight: bold;
            }
            .fail {
                color: red;
                font-weight: bold;
            }
            .qa-item {
                margin-bottom: 20px;
                padding: 15px;
                border-radius: 5px;
                border-left: 5px solid #3498db;
                background-color: #fff;
                box-shadow: 0 1px 3px rgba(0,0,0,0.1);
            }
            .qa-question {
                font-weight: bold;
                font-size: 18px;
                margin-bottom: 10px;
                color: #2980b9;
            }
            .qa-answer {
                margin-bottom: 10px;
                line-height: 1.8;
            }
            .qa-calculation {
                font-family: monospace;
                background-color: #f8f8f8;
                padding: 8px;
                border-radius: 4px;
                border: 1px solid #ddd;
                font-size: 14px;
                color: #333;
                overflow-x: auto;
            }
            .tabs {
                overflow: hidden;
                background-color: #f1f1f1;
                border-radius: 5px 5px 0 0;
            }
            .tab-button {
                background-color: inherit;
                float: left;
                border: none;
                outline: none;
                cursor: pointer;
                padding: 14px 16px;
                transition: 0.3s;
                font-size: 17px;
            }
            .tab-button:hover {
                background-color: #ddd;
            }
            .tab-button.active {
                background-color: #3498db;
                color: white;
            }
            .tab-content {
                display: none;
                padding: 20px;
                border: 1px solid #ccc;
                border-top: none;
                border-radius: 0 0 5px 5px;
            }
        </style>
    </head>"""

    def _get_page_start(self):
        """
        Get the beginning of the page body.

        Returns:
            Beginning of page body as string
        """
        return """
    <body>
        <h1>Synthetic Biology Dataset Report</h1>"""

    def _get_dataset_overview(self, dataset_validation, answer_validation):
        """
        Get the dataset overview section.

        Args:
            dataset_validation: Results of dataset pattern validation
            answer_validation: Results of answer validation

        Returns:
            Dataset overview section as string
        """
        return f"""
        <div class="section">
            <h2>Dataset Overview</h2>
            <div class="summary">
                <div class="summary-item">
                    <div class="summary-value">{self.df.shape[0]:,}</div>
                    <div class="summary-label">Total Records</div>
                </div>
                <div class="summary-item">
                    <div class="summary-value">{self.df.shape[1]}</div>
                    <div class="summary-label">Fields/Columns</div>
                </div>
                <div class="summary-item">
                    <div class="summary-value">{dataset_validation['summary']['pass_rate']}%</div>
                    <div class="summary-label">Pattern Validation</div>
                </div>
                <div class="summary-item">
                    <div class="summary-value">{answer_validation['summary']['pass_rate']}%</div>
                    <div class="summary-label">Answer Validation</div>
                </div>
            </div>"""

    def _get_tab_headers(self):
        """
        Get the tab navigation headers.

        Returns:
            Tab navigation HTML as string
        """
        return """
            <div class="tabs">
                <button class="tab-button active" onclick="openTab(event, 'dataset-info')">Dataset Info</button>
                <button class="tab-button" onclick="openTab(event, 'pattern-validation')">Pattern Validation</button>
                <button class="tab-button" onclick="openTab(event, 'answer-validation')">Answer Validation</button>
                <button class="tab-button" onclick="openTab(event, 'qa-document')">Q&A Document</button>
            </div>"""

    def _get_dataset_info_tab(self):
        """
        Get the dataset info tab content.

        Returns:
            Dataset info tab HTML as string
        """
        # Format the list of unique animal types
        animal_types = ', '.join(sorted(self.df.animal_type.unique()))

        # Format the list of unique colors
        colors = ', '.join(sorted(self.df.color.unique()))

        # Format weight range
        weight_min = round(self.df.weight_kg.min(), 1)
        weight_max = round(self.df.weight_kg.max(), 1)

        # Format height range
        height_min = round(self.df.height_cm.min(), 1)
        height_max = round(self.df.height_cm.max(), 1)

        return f"""
            <div id="dataset-info" class="tab-content" style="display: block;">
                <h3>Basic Dataset Statistics</h3>
                <table>
                    <tr>
                        <th>Field</th>
                        <th>Type</th>
                        <th>Sample Values</th>
                        <th>Description</th>
                    </tr>
                    <tr>
                        <td>animal_type</td>
                        <td>categorical</td>
                        <td>{animal_types}</td>
                        <td>Type of animal, evenly distributed</td>
                    </tr>
                    <tr>
                        <td>weight_kg</td>
                        <td>float</td>
                        <td>Range: {weight_min} - {weight_max}</td>
                        <td>Weight in kg with type-specific means</td>
                    </tr>
                    <tr>
                        <td>height_cm</td>
                        <td>float</td>
                        <td>Range: {height_min} - {height_max}</td>
                        <td>Height in cm, negatively correlated with weight</td>
                    </tr>
                    <tr>
                        <td>birth_date</td>
                        <td>datetime</td>
                        <td>All in winter months (Dec, Jan, Feb)</td>
                        <td>Birth date with seasonal pattern (winter only)</td>
                    </tr>
                    <tr>
                        <td>color</td>
                        <td>categorical</td>
                        <td>{colors}</td>
                        <td>Color with strong correlation to animal type</td>
                    </tr>
                    <tr>
                        <td>can_swim</td>
                        <td>boolean</td>
                        <td>True/False</td>
                        <td>Cats and birds can swim (~90%), others cannot (~10%)</td>
                    </tr>
                    <tr>
                        <td>can_run</td>
                        <td>boolean</td>
                        <td>True/False</td>
                        <td>Dogs and birds: age-dependent pattern</td>
                    </tr>
                    <tr>
                        <td>watches_youtube</td>
                        <td>boolean</td>
                        <td>True/False</td>
                        <td>Fish and turtles watch (~90%), others do not (~10%)</td>
                    </tr>
                </table>
            </div>"""

    def _get_validation_tab(self, validation):
        """
        Get the pattern validation tab content.

        Args:
            validation: Results of pattern validation

        Returns:
            Pattern validation tab HTML as string
        """
        # Generate the table rows for each test
        table_rows = []
        for test in validation['tests']:
            # Determine status display and CSS class
            status_display = "PASS" if test['passed'] else "FAIL"
            status_class = "pass" if test['passed'] else "fail"

            # Create the row
            row = f"""
                    <tr>
                        <td>{test['name']}</td>
                        <td>{test['actual']}</td>
                        <td>{test['expected']}</td>
                        <td class="{status_class}">{status_display}</td>
                        <td>{test['description']}</td>
                    </tr>"""
            table_rows.append(row)

        # Join all rows
        all_rows = ''.join(table_rows)

        return f"""
            <div id="pattern-validation" class="tab-content">
                <h3>Dataset Pattern Validation</h3>
                <div class="summary">
                    <div class="summary-item">
                        <div class="summary-value">{validation['summary']['pass_count']}</div>
                        <div class="summary-label">Tests Passed</div>
                    </div>
                    <div class="summary-item">
                        <div class="summary-value">{validation['summary']['fail_count']}</div>
                        <div class="summary-label">Tests Failed</div>
                    </div>
                    <div class="summary-item">
                        <div class="summary-value">{validation['summary']['total_tests']}</div>
                        <div class="summary-label">Total Tests</div>
                    </div>
                    <div class="summary-item">
                        <div class="summary-value">{validation['summary']['pass_rate']}%</div>
                        <div class="summary-label">Pass Rate</div>
                    </div>
                </div>

                <table>
                    <tr>
                        <th>Test</th>
                        <th>Actual</th>
                        <th>Expected</th>
                        <th>Status</th>
                        <th>Description</th>
                    </tr>{all_rows}
                </table>
            </div>"""

    def _get_answer_validation_tab(self, validation):
        """
        Get the answer validation tab content.

        Args:
            validation: Results of answer validation

        Returns:
            Answer validation tab HTML as string
        """
        # Generate table rows for answer validation
        table_rows = []
        for test in validation['tests']:
            # Determine status display and CSS class
            status_display = "PASS" if test['passed'] else "FAIL"
            status_class = "pass" if test['passed'] else "fail"

            # Get the answer excerpt, defaulting to N/A if not available
            answer_excerpt = test.get('answer_excerpt', 'N/A')

            # Create the row
            row = f"""
                    <tr>
                        <td>{test['name']}</td>
                        <td>{test['expected']}</td>
                        <td class="{status_class}">{status_display}</td>
                        <td>{answer_excerpt}</td>
                    </tr>"""
            table_rows.append(row)

        # Join all rows
        all_rows = ''.join(table_rows)

        return f"""
            <div id="answer-validation" class="tab-content">
                <h3>Answer Validation</h3>
                <div class="summary">
                    <div class="summary-item">
                        <div class="summary-value">{validation['summary']['pass_count']}</div>
                        <div class="summary-label">Tests Passed</div>
                    </div>
                    <div class="summary-item">
                        <div class="summary-value">{validation['summary']['fail_count']}</div>
                        <div class="summary-label">Tests Failed</div>
                    </div>
                    <div class="summary-item">
                        <div class="summary-value">{validation['summary']['total_tests']}</div>
                        <div class="summary-label">Total Tests</div>
                    </div>
                    <div class="summary-item">
                        <div class="summary-value">{validation['summary']['pass_rate']}%</div>
                        <div class="summary-label">Pass Rate</div>
                    </div>
                </div>

                <table>
                    <tr>
                        <th>Question</th>
                        <th>Expected</th>
                        <th>Status</th>
                        <th>Answer Excerpt</th>
                    </tr>{all_rows}
                </table>
            </div>"""

    def _get_qa_tab(self):
        """
        Get the Q&A document tab content.

        Returns:
            Q&A document tab HTML as string
        """
        # Helper function to create Q&A HTML for a category
        def create_qa_section(question_keys):
            """Create HTML for a group of Q&A items"""
            items = []
            for key in question_keys:
                if key in self.answers:
                    qa = self.answers[key]
                    item = f"""
                    <div class="qa-item">
                        <div class="qa-question">{qa['question']}</div>
                        <div class="qa-answer"><strong>Answer:</strong> {qa['answer']}</div>
                        <div class="qa-calculation"><strong>Calculation:</strong> {qa['calculation']}</div>
                    </div>"""
                    items.append(item)
            return ''.join(items)

        # Define question categories
        basic_stat_keys = ['most_common_animal_type', 'average_weight', 'average_cat_weight',
                        'heaviest_animal_type', 'bird_count', 'average_friends', 'most_friends']

        categorical_keys = ['most_common_color', 'color_animal_relation', 'red_animals_swim',
                        'blue_vs_green_weight']

        time_keys = ['birth_season', 'birth_pattern', 'q1_births']

        correlation_keys = ['weight_height_correlation', 'food_prediction', 'age_friends_relation']

        boolean_keys = ['fly_percentage', 'swim_types', 'run_age_effect',
                    'youtube_percentage', 'swim_fly_combo']

        complex_keys = ['height_popularity']

        text_keys = ['film_social_media', 'animal_sentiment']

        # Create the HTML sections
        basic_stats_html = create_qa_section(basic_stat_keys)
        categorical_html = create_qa_section(categorical_keys)
        time_series_html = create_qa_section(time_keys)
        correlation_html = create_qa_section(correlation_keys)
        boolean_html = create_qa_section(boolean_keys)
        complex_html = create_qa_section(complex_keys)
        text_html = create_qa_section(text_keys)

        return f"""
            <div id="qa-document" class="tab-content">
                <h3>Questions and Answers</h3>

                <h4>Basic Statistics Questions</h4>
                {basic_stats_html}

                <h4>Categorical Analysis Questions</h4>
                {categorical_html}

                <h4>Time Series Analysis Questions</h4>
                {time_series_html}

                <h4>Correlation and Relationship Questions</h4>
                {correlation_html}

                <h4>Boolean Pattern Questions</h4>
                {boolean_html}

                <h4>Complex Pattern Questions</h4>
                {complex_html}

                <h4>Text Analysis Questions</h4>
                {text_html}
            </div>"""

    def _get_page_end(self):
        """
        Get the page end section with JavaScript.

        Returns:
            Page end HTML as string
        """
        return """
        </div>

        <script>
        function openTab(evt, tabName) {
            var i, tabcontent, tablinks;
            tabcontent = document.getElementsByClassName("tab-content");
            for (i = 0; i < tabcontent.length; i++) {
                tabcontent[i].style.display = "none";
            }
            tablinks = document.getElementsByClassName("tab-button");
            for (i = 0; i < tablinks.length; i++) {
                tablinks[i].className = tablinks[i].className.replace(" active", "");
            }
            document.getElementById(tabName).style.display = "block";
            evt.currentTarget.className += " active";
        }
        </script>
    </body>
    </html>"""


def zip_output_directory(directory_path: str) -> str:
    """
    Create a zip archive of the specified directory.

    Args:
        directory_path: Path to the directory to be zipped

    Returns:
        Path to the created zip file

    Raises:
        FileNotFoundError: If the directory doesn't exist
        OSError: If there's an error during zip creation
    """
    import zipfile
    import os
    import logging

    logger = logging.getLogger(__name__)

    if not os.path.exists(directory_path):
        raise FileNotFoundError(f"Directory not found: {directory_path}")

    # Create zip filename based on directory name
    zip_filename = f"{directory_path}.zip"

    try:
        logger.info(f"Creating zip archive of {directory_path}...")

        with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
            # Walk through all files and directories
            for root, _, files in os.walk(directory_path):
                for file in files:
                    # Get the full file path
                    file_path = os.path.join(root, file)

                    # Calculate the relative path for the archive
                    relative_path = os.path.relpath(file_path, os.path.dirname(directory_path))

                    # Add the file to the zip
                    zipf.write(file_path, relative_path)

        logger.info(f"Zip archive created: {zip_filename}")
        return zip_filename

    except Exception as e:
        logger.error(f"Error creating zip archive: {str(e)}")
        raise OSError(f"Failed to create zip archive: {str(e)}")

def main(n_rows: int = 100, random_seed: int = 42) -> Dict[str, str]:
    """
    Generate a synthetic biology dataset, validate it, and create comprehensive documentation.

    Args:
        n_rows: Number of rows to generate in the dataset
        random_seed: Seed for reproducible random number generation

    Returns:
        Dictionary with paths to all generated files
    """
    # Create timestamp for filenames
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Make output directory if it doesn't exist
    output_dir = f"synthetic_biology_data_{timestamp}"
    os.makedirs(output_dir, exist_ok=True)

    # Generate and validate the dataset
    generator = SyntheticDataGenerator(n_rows, random_seed)
    df = generator.generate()

    # Calculate answers to questions
    answers = generator.calculate_answers()

    # Save dataset
    csv_filename = os.path.join(output_dir, f"synthetic_biology_dataset.csv")
    df.to_csv(csv_filename, index=False)

    # Generate validation report
    dataset_validation = generator.validate_dataset()
    answers_validation = generator.validate_answers()

    # Save validation data as JSON
    validation_json_filename = os.path.join(output_dir, f"validation_results.json")
    with open(validation_json_filename, "w") as f:
        json.dump({
            "dataset_validation": dataset_validation,
            "answers_validation": answers_validation
        }, f, indent=2, default=str)

    # Generate comprehensive HTML report
    html_report_filename = os.path.join(output_dir, f"dataset_report.html")
    generator.generate_html_report(html_report_filename)

    # Save answers as JSON
    answers_json_filename = os.path.join(output_dir, f"qa_answers.json")
    with open(answers_json_filename, "w") as f:
        json.dump(answers, f, indent=2, default=str)

    logger.info(f"Process complete. All files saved to {output_dir}/")



    return {
        "output_directory": output_dir,
        "dataset_csv": csv_filename,
        "validation_json": validation_json_filename,
        "html_report": html_report_filename,
        "answers_json": answers_json_filename,
    }




if __name__ == "__main__":
    result = main(20000)
    print(f"\nProcess completed successfully!")
    print(f"Generated files:")
    print(f"- Dataset: {result['dataset_csv']}")
    print(f"- Validation Results: {result['validation_json']}")
    print(f"- HTML Report: {result['html_report']}")
    print(f"- Q&A Answers: {result['answers_json']}")
    print(f"\nOpen {result['html_report']} in a web browser to view the complete report.")
    # Create a zip file of the output directory
    zip_file = zip_output_directory(result["output_directory"])



<ipython-input-2-fb5280a4b98a>:560: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  color_pct = round(self.df[self.df.animal_type == animal][self.df.color == expected_color].shape[0] /
<ipython-input-2-fb5280a4b98a>:560: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  color_pct = round(self.df[self.df.animal_type == animal][self.df.color == expected_color].shape[0] /
<ipython-input-2-fb5280a4b98a>:560: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  color_pct = round(self.df[self.df.animal_type == animal][self.df.color == expected_color].shape[0] /
<ipython-input-2-fb5280a4b98a>:560: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  color_pct = round(self.df[self.df.animal_type == animal][self.df.color == expected_color].shape[0] /
<ipython-input-2-fb5280a4b98a>:560: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  color_pct = round(self.df[


Process completed successfully!
Generated files:
- Dataset: synthetic_biology_data_20250422_014229/synthetic_biology_dataset.csv
- Validation Results: synthetic_biology_data_20250422_014229/validation_results.json
- HTML Report: synthetic_biology_data_20250422_014229/dataset_report.html
- Q&A Answers: synthetic_biology_data_20250422_014229/qa_answers.json

Open synthetic_biology_data_20250422_014229/dataset_report.html in a web browser to view the complete report.
